In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from collections import Counter

#Iris Dataset
df = pd.read_csv(u'iris.txt',sep=' ')


In [2]:
#k nearest
k = 20 
#Dataset Matrix
A = np.hstack([
        np.matrix(df.sl).T, 
        np.matrix(df.sw).T, 
        np.matrix(df.pl).T, 
        np.matrix(df.pw).T])
#Classes
c = np.matrix(df.c).T

### Distance Array:

In [3]:
#sortedDistanceArray returns the sorted array that holds the distances between A and x(x_test) 
#and the class of the iris 
def sortedDistanceArray(A,x):
    
    N = A.shape[0]
    D = np.zeros((N,2))
    
    for i in range(N):
        e = A[i] - x
        D[i,0] =  (np.sum(e*e.T))**(1/2.0) #square root for euclidean distance
        D[i,1] = (int(c[i])) #holds the iris class 
     
    return sorted(D, key=lambda entry: entry[0]) 


### Example: Finding the class of 51st element of the matrix A:

In [4]:
#example: x_test is the 51st element of A matrix
x_test = A[50]
D = sortedDistanceArray(A, x_test)[:k] #first k element in sorted distance array
D

[array([ 0.,  2.]),
 array([ 0.26457513,  2.        ]),
 array([ 0.33166248,  2.        ]),
 array([ 0.43588989,  2.        ]),
 array([ 0.45825757,  2.        ]),
 array([ 0.51961524,  2.        ]),
 array([ 0.53851648,  2.        ]),
 array([ 0.55677644,  2.        ]),
 array([ 0.64031242,  2.        ]),
 array([ 0.65574385,  2.        ]),
 array([ 0.73484692,  2.        ]),
 array([ 0.78740079,  2.        ]),
 array([ 0.87749644,  3.        ]),
 array([ 0.90553851,  3.        ]),
 array([ 0.92736185,  2.        ]),
 array([ 0.9486833,  2.       ]),
 array([ 0.9486833,  3.       ]),
 array([ 0.9486833,  2.       ]),
 array([ 0.96953597,  3.        ]),
 array([ 0.98488578,  3.        ])]

### The iris class of 51st element:

In [5]:
#count the most common class in the array
u, indices = np.unique(D, return_inverse=True)
u[np.argmax(np.bincount(indices))]

2.0

In [ ]:
## Use a heap to store the smallest items

import heapq as hp

# Define an object and overload custom comparison operators
class tup:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val,self.idx)
    
        
## Search for the nearest K   
K = 20
heap = []
N = A.shape[0]   

for j in range(N):
    heap = []
    ## Fill in the heap with dummy nodes
    for k in range(K):
        hp.heappush(heap, tup(np.inf, -1))

    for i in range(N):
        if i != j:
            e = A[i,:]- A[j,:]
            e = e.reshape((4,1))
            tp = tup(float(e.T*e), i)
            if tp <= heap[0]:
                hp.heapreplace(heap, tp)

    for t in range(len(heap)):
        h = hp.heappop(heap)
        print(h, float(c[h.idx]))
    print('--'*5)